# Part 5: projects - Check inventory of libraries

### Context
Collection managers need to know the collection status of other libraries in order to guide their acquisition and weeding policies. The goal here is to build a script that uses an ISBN or ISSN to check which libraries have a printed inventory of the document.


### To do

The system will search for records at the NZ level, considering only fields 020a and 022a, and then, based on the records found, it will retrieve the list of IZs and libraries that own the document.

### Result
The result is a dictionary with IZ codes as keys and a set of library codes that own the document as values. 

```
{
    'IZ_code_1': {'lib_code_1', 'lib_code_2'},
    'IZ_code_2': {'lib_code_3'},
    ...
}
```
It is possible to use lists instead of a set, but you will have duplicated libraries in the list.

### Difficulty level
Medium

In [5]:
# Import libraries
from almasru.client import SruClient, SruRecord, IzSruRecord, SruRequest
from almasru.utils import check_removable_records, analyse_records
from almasru import dedup
from almasru.briefrecord import BriefRecFactory, BriefRec
from almasru import config_log
from lxml import etree

parser = etree.XMLParser(remove_blank_text=True, remove_comments=True, ns_clean=True)

ns = {
    "srw": "http://www.loc.gov/zing/srw/",
    "marc": "http://www.loc.gov/MARC21/slim"
}

# Config logs
config_log()
zone = '41SLSP_NETWORK'
SruClient.set_base_url(f'https://swisscovery.ch/view/sru/{zone}')
std_num = '0020-7985'
records = SruRequest(f'alma.standard_number={std_num}').records

def has_record_std_num_in_02x(std_num, record):
    std_nums = [f.text.replace('-', '') for f in record.data.findall('.//marc:datafield[@tag="020"]/marc:subfield[@code="a"]', namespaces=ns)]
    std_nums += [f.text.replace('-', '') for f in record.data.findall('.//marc:datafield[@tag="022"]/marc:subfield[@code="a"]', namespaces=ns)]
    if std_num.replace('-', '') in std_nums:
        return True
    else:
        return False

records = [r for r in records if has_record_std_num_in_02x(std_num, r)]

result = dict()
for record in records:
    izs = record.get_iz_using_rec()
    for iz in izs:
        result[iz] = set()
        iz_rec = record.get_iz_record(f'https://swisscovery.ch/view/sru/{iz}')
        inventory = iz_rec.get_inventory_info()
        for hol in inventory:
            result[iz].add(hol['library'])

2025-09-02 08:29:04,855 - INFO - Records 1 - 9 / 9, "alma.standard_number=0020-7985": 9


4

2025-09-02 08:29:06,654 - INFO - SruRecord('991170911588205501', base_url='https://swisscovery.ch/view/sru/41SLSP_NETWORK'): record used in IZ: 41BIG_INST
2025-09-02 08:29:06,658 - INFO - SruRecord('991170911588205501', base_url='https://swisscovery.ch/view/sru/41SLSP_NETWORK'): 1 records in IZ found
2025-09-02 08:29:06,661 - INFO - IzSruRecord('993900804101791', base_url='https://swisscovery.ch/view/sru/41BIG_INST'): inventory in IZ found (2)
2025-09-02 08:29:06,664 - INFO - SruRecord('991124328919705501', base_url='https://swisscovery.ch/view/sru/41SLSP_NETWORK'): record used in IZ: 41SLSP_RZH, 41SLSP_SBK, 41SLSP_UBS
2025-09-02 08:29:06,666 - INFO - SruRecord('991124328919705501', base_url='https://swisscovery.ch/view/sru/41SLSP_NETWORK'): 3 records in IZ found
2025-09-02 08:29:06,668 - INFO - IzSruRecord('990009912310205526', base_url='https://swisscovery.ch/view/sru/41SLSP_RZH'): inventory in IZ found (2)
2025-09-02 08:29:06,670 - INFO - SruRecord('991124328919705501', base_url='ht

In [8]:
result

{'41BIG_INST': {'BIG'},
 '41SLSP_RZH': {'E19'},
 '41SLSP_SBK': {'UZSBK', 'Z0SBK'},
 '41SLSP_UBS': {'A125'},
 '41SLSP_UGE': {'uge_mail'},
 '41SLSP_UNE': {'une_sfm'},
 '41SLSP_IID': {'iid'},
 '41SLSP_VGE': {'vge_bge'},
 '41SLSP_BCUFR': {'BCUF_BP2'}}

In [20]:
std_fields = [f.text.replace('-', '') for f in record.data.findall('.//marc:datafield[@tag="020"]/marc:subfield[@code="a"]', namespaces=ns)]
std_fields += [f.text.replace('-', '') for f in record.data.findall('.//marc:datafield[@tag="022"]/marc:subfield[@code="a"]', namespaces=ns)]

In [21]:
std_fields

['00207985']

In [19]:
record = records[3]

In [18]:
print(records[3])

<record xmlns="http://www.loc.gov/MARC21/slim">
  <leader>02283nas a2200433 c 4500</leader>
  <controlfield tag="001">991071589009705501</controlfield>
  <controlfield tag="005">20250731110615.0</controlfield>
  <controlfield tag="008">001117d19671992sz || p|  ||| 0    0eng d</controlfield>
  <datafield ind1=" " ind2=" " tag="022">
    <subfield code="a">0020-7985</subfield>
  </datafield>
  <datafield ind1=" " ind2=" " tag="035">
    <subfield code="a">(swissbib)081553137-41slsp_network</subfield>
  </datafield>
  <datafield ind1=" " ind2=" " tag="035">
    <subfield code="a">081553137</subfield>
    <subfield code="9">ExL</subfield>
  </datafield>
  <datafield ind1=" " ind2=" " tag="035">
    <subfield code="a">(RERO)0814503-41slsp</subfield>
  </datafield>
  <datafield ind1=" " ind2=" " tag="040">
    <subfield code="a">RERO geufpe</subfield>
    <subfield code="d">CH-ZuSLS</subfield>
  </datafield>
  <datafield ind1="0" ind2=" " tag="041">
    <subfield code="a">eng</subfield>
    